# Convergence Benchmarks

In [2]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from quantum_tools import *
from experimental import *
from spam import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Fit SPAM model

### Generate True Model, Full POVM and Inital

In [9]:
def generate_spam_benchmark(n=3, c1=1, c2=1):
    d = 2**n

    init_target = InitialState(d, c=c1)
    povm_target = POVM(d, c=c2)

    spam_target = SPAM(init = init_target,
                       povm = povm_target)
    
    return spam_target


def generate_spam_data(spam_target, N_spam=None, shots=1024):
    n = int(np.log2(spam_target.d))
    inputs_spam, _ = generate_pauliInput_circuits(n)
    N_spam = inputs_spam.shape[0]

    state = tf.repeat(spam_target.init.init[None,:,:], N_spam, axis=0)
    state = apply_unitary(state, inputs_spam)
    targets_spam = measurement(state, povm = spam_target.povm.povm)

    #add noise
    if shots != 0:
        targets_spam = add_shot_noise(targets_spam, shots=shots)
        
    return inputs_spam, targets_spam


def spam_pipeline(n, 
                  c1, 
                  c2,
                  shots,
                  use_corr = False,
                  num_iter = 2000):
    spam_target = generate_spam_benchmark(n=n, c1=c1, c2=c2)
    inputs_spam, targets_spam = generate_spam_data(spam_target, N_spam=None, shots=shots)

    
    if use_corr:
        povm = CorruptionMatrix(d, c=0.9)
    else:
        povm = POVM(d, c=0.9)
        
    spam_model = SPAM(init = InitialState(d, c=0.9),
                      povm = povm,
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
        

    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = num_iter,
                     verbose = False,
                    )
    
    return spam_target, spam_model


def povm_fidelity(povm_a, povm_b):
    d = povm_a.shape[0]
    ab = tf.matmul(povm_a, povm_b)
    ab_sqrt = tf.linalg.sqrtm(ab)
    fidelity = tf.math.reduce_sum(tf.linalg.trace(ab_sqrt))/d
    return fidelity

## Three Qubit

In [10]:
n = 3
d = 2**n
c1 = 0.8
c2 = 0.8


num_reps = 10
shots_list = np.linspace(7, 16, 10)
shots_list = 2**shots_list
shots_list = np.append(shots_list, 0)

init_fid_list = np.zeros((len(shots_list), num_reps))
povm_fid_list = np.zeros((len(shots_list), num_reps))

for i, shots in enumerate(tqdm(shots_list)):   
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)
    for rep in range(num_reps):    
        spam_model, spam_target = spam_pipeline(n=n,
                                                c1=c1,
                                                c2=c2,
                                                shots = shots,
                                                use_corr=False,
                                                num_iter = 3000,
                                                )
        
        init_fid_list[i, rep] = state_fidelity(spam_model.init.init, spam_target.init.init)
        povm_fid_list[i, rep] = povm_fidelity(spam_model.povm.povm, spam_target.povm.povm)
        
#saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_3qubit_full.data"))
saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_3qubit_full_shotnoise.data"))

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

0.005213434651638776


C:\Users\krist\anaconda3\envs\env_qiskit\lib\site-packages\tensorflow\python\framework\ops.py:1064: ComplexWarning: Casting complex values to real discards the imaginary part
  return float(self._numpy())


  0%|          | 0/3000 [00:00<?, ?it/s]

...

1.2452731788268e-07


  0%|          | 0/3000 [00:00<?, ?it/s]

1.1366041582126645e-07


  0%|          | 0/3000 [00:00<?, ?it/s]

7.645412629900104e-08


In [11]:
n = 3
d = 2**n
c1 = 0.8
c2 = 0.8


num_reps = 10
shots_list = np.linspace(7, 16, 10)
shots_list = 2**shots_list
shots_list = np.append(shots_list, 0)

init_fid_list = np.zeros((len(shots_list), num_reps))
povm_fid_list = np.zeros((len(shots_list), num_reps))

for i, shots in enumerate(tqdm(shots_list)):   
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)
    for rep in range(num_reps):    
        spam_model, spam_target = spam_pipeline(n=n,
                                                c1=c1,
                                                c2=c2,
                                                shots = shots,
                                                use_corr=True,
                                                num_iter = 3000,
                                                )
        
        init_fid_list[i, rep] = state_fidelity(spam_model.init.init, spam_target.init.init)
        povm_fid_list[i, rep] = povm_fidelity(spam_model.povm.povm, spam_target.povm.povm)
        
#saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_3qubit_full.data"))
saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_3qubit_corr_shotnoise.data"))

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

0.006674605703158114


  0%|          | 0/3000 [00:00<?, ?it/s]

0.006107021993884357


...

0.00018261288355182772


  0%|          | 0/3000 [00:00<?, ?it/s]

0.00020857988436405738


  0%|          | 0/3000 [00:00<?, ?it/s]

0.00021810361228769313


## Four Qubit

In [13]:
n = 4
d = 2**n
c1 = 0.8
c2 = 0.8


num_reps = 10
shots_list = np.linspace(7, 16, 10)
shots_list = 2**shots_list
shots_list = np.append(shots_list, 0)

init_fid_list = np.zeros((len(shots_list), num_reps))
povm_fid_list = np.zeros((len(shots_list), num_reps))

for i, shots in enumerate(tqdm(shots_list)):   
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)
    for rep in range(num_reps):    
        spam_model, spam_target = spam_pipeline(n=n,
                                                c1=c1,
                                                c2=c2,
                                                shots = shots,
                                                use_corr=False,
                                                num_iter = 3000,
                                                )
        
        init_fid_list[i, rep] = state_fidelity(spam_model.init.init, spam_target.init.init)
        povm_fid_list[i, rep] = povm_fidelity(spam_model.povm.povm, spam_target.povm.povm)
        
#saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_3qubit_full.data"))
saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_4qubit_full_shotnoise.data"))

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

0.006071236438589084


  0%|          | 0/3000 [00:00<?, ?it/s]

0.005881143982828569


...

1.9788068766474725e-07


  0%|          | 0/3000 [00:00<?, ?it/s]

1.9014713753522951e-07


  0%|          | 0/3000 [00:00<?, ?it/s]

2.2853466790403343e-07


In [ ]:
n = 4
d = 2**n
c1 = 0.8
c2 = 0.8


num_reps = 10
shots_list = np.linspace(7, 16, 10)
shots_list = 2**shots_list
shots_list = np.append(shots_list, 0)

init_fid_list = np.zeros((len(shots_list), num_reps))
povm_fid_list = np.zeros((len(shots_list), num_reps))

for i, shots in enumerate(tqdm(shots_list)):   
    np.random.seed(42)
    random.seed(42)
    tf.random.set_seed(42)
    for rep in range(num_reps):    
        spam_model, spam_target = spam_pipeline(n=n,
                                                c1=c1,
                                                c2=c2,
                                                shots = shots,
                                                use_corr=True,
                                                num_iter = 3000,
                                                )
        
        init_fid_list[i, rep] = state_fidelity(spam_model.init.init, spam_target.init.init)
        povm_fid_list[i, rep] = povm_fidelity(spam_model.povm.povm, spam_target.povm.povm)
        
#saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_3qubit_full.data"))
saver([init_fid_list, povm_fid_list], data_path("spam_fidelity_4qubit_corr_shotnoise.data"))

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

0.007054109768718516


  0%|          | 0/3000 [00:00<?, ?it/s]

0.006832462555219922


...

  0%|          | 0/3000 [00:00<?, ?it/s]

9.815369376893329e-05


  0%|          | 0/3000 [00:00<?, ?it/s]

9.30498219774997e-05


  0%|          | 0/3000 [00:00<?, ?it/s]